In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/monitor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Функция, определяющая, является ли строка пустой
def isBlank (string):
    if string and string.strip():
        return False
    return True


In [4]:
import re

patterns = "[A-Za-z0-9(){}–«»,:;\"'.“”!#$%&<=>?@\xa0/№-]+"

texts = []

# В цикле читаются все документы из папки ./documents, из текста каждого документа убираются спец. символы,
# указанные в переменной patterns
# После этого тексты всех документов собирается в один массив
for i in range(1,101):
    path='./documents/out{num}.txt'.format(num=i)
    with open(path,'r') as file:
        text = ''
        for line in file.readlines():
            if not isBlank(line):
                line = re.sub(patterns, ' ', line)
                text += ' ' + line
        text = text.replace('\n', ' ')
        texts.append(text)

# print(texts)

In [7]:
tokens = set()
lemmas = set()

# чтение терминов из файла с терминами
with open('./tokens.txt','r') as file:
    for line in file.readlines():
        tokens.add(line.strip())

# чтение лемм из файла с леммами    
with open('./lemmas.txt','r') as file:
    for line in file.readlines():
        lemmas.add(line.split(":")[0].strip())
    
print(len(tokens), ' ', len(lemmas))

8658   5314


In [9]:
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

# функция для подсчета tf для терминов и лемм
# в цикле идет проход по текстам всех документов, для каждого документа в два словаря 
# добавляется или обновляется информация о каждом термине и каждой лемме(номер документа и tf в документе)


def count_tf(texts):
    lms_tf = {}
    tks_tf = {}
    for i in range(0,len(texts)):
        words = texts[i].split(' ')
        tklen = 0
        for word in words:
            token = word.lower()
            if token and token not in stopwords_ru:
                token = token.strip()
                
                if token in tokens:
                    if token in tks_tf.keys():
                        if i+1 in tks_tf[token].keys():
                            tks_tf[token][i+1] += 1
                        else:
                            tks_tf[token][i+1] = 1
                    else:
                        tks_tf[token] = {i+1:1}
                    tklen += 1
                
                
                nor_token = morph.normal_forms(token)[0]
                if nor_token in lemmas:
                    if nor_token in lms_tf.keys():
                        if i+1 in lms_tf[nor_token].keys():
                            lms_tf[nor_token][i+1] += 1
                        else:
                            lms_tf[nor_token][i+1] = 1
                    else:
                        lms_tf[nor_token] = {i+1:1}
        
        for t in tks_tf.keys():
            if i+1 in tks_tf[t].keys():
                tks_tf[t][i+1] /= len(words)
        
        for l in lms_tf.keys():
            if i+1 in lms_tf[l].keys():
                lms_tf[l][i+1] /= tklen
    
    return tks_tf,lms_tf
                    
                
tks_tf,lms_tf = count_tf(texts)

# tks_tf,lms_tf - словари формата 
# {слово: {номер_документа: tf, номер_документа: tf, ...}, 
# слово: {номер_документа: tf, номер_документа: tf, ...} ...}

In [30]:
import math


# получение данных для результирующих файлов
# для каждого слова в словарях tks_tf,lms_tf в результирующий словарь 
# по ключу документа добавляется информация вида слово: [tf, tf*idf]
# tks_res, lms_res - словари вида 
# {номер_документа: {слово:[idf, tf*idf], слово:[idf, tf*idf], ...},
# номер_документа: {слово:[idf, tf*idf], слово:[idf, tf*idf], ...} ...}

tks_res = {}

for key in tks_tf.keys():
    idf = math.log10(100/len(tks_tf[key].keys()))
    
    for k in tks_tf[key].keys():
        if k in tks_res.keys():
            tks_res[k][key] = [idf, idf*tks_tf[key][k]]
        else:
            tks_res[k] = {key: [idf, idf*tks_tf[key][k]]}

lms_res = {}

for key in lms_tf.keys():
    idf = math.log10(100/len(lms_tf[key].keys()))
    
    for k in lms_tf[key].keys():
        if k in lms_res.keys():
            lms_res[k][key] = [idf, idf*lms_tf[key][k]]
        else:
            lms_res[k] = {key: [idf, idf*lms_tf[key][k]]}
            


In [36]:
# запись выходных данных в файлы

for num in tks_res.keys():
    with open('./result_tokens/outt{}.txt'.format(num),'w') as t_file:
        for word in tks_res[num].keys():
            t_file.write('{} {} {}\n'.format(word,tks_res[num][word][0],tks_res[num][word][1]))
    
    with open('./result_lemmas/outl{}.txt'.format(num),'w') as l_file:
        for word in lms_res[num].keys():
            l_file.write('{} {} {}\n'.format(word,lms_res[num][word][0],lms_res[num][word][1]))
